In [1]:
import time

import asyncio
from math import radians, cos, sqrt, atan2, pi, degrees

# nest_asyncio allows async operations within the Juypeter Notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
49.69440585066409, 10.827450473363868

position =  ("49.69440585066409","10.827450473363868" ) #Home orkshop
#destination = ("49.69848278709498","10.85355993252834" )  #HBurkhardtome
destination = ("49.684309507663706", "10.837448319639334") #Krausenbechhofen
#destination = ("49.694428996515875", "10.827444023181") # Fence Lin
#destination=("49.694407742568984", "10.827534547767414") # Kellerstrasse at Workshop




In [3]:
def normalize(angle):
    '''Normalize degrees to -PI to PI radians range'''
    return (angle + pi) % (2 * pi) - pi

In [4]:
def translateValue(valueIn,minIn,maxIn,minOut,maxOut):
    ''' Translate value from one range to another '''
    valueIn = min(maxIn, max(minIn, valueIn)) # clamp the input value
    return (valueIn-minIn)/(maxIn-minIn)*(maxOut-minOut)+minOut # translate the value
    

In [5]:
def convert_dd_int(degreedecimal:str) -> int:
    '''
    converts degree decimal to big int
    "49.6939697" => 496939697
    '''
    degree, decimal = degreedecimal.split('.')
    dd = str(degree)+str(decimal) 
    dd = '{:<09s}'.format(dd) # Micropython 10 Digits Precision (2147483647), we choose 9 digit precision
    dd = dd[:9]
    return int(dd)

In [6]:
def positionDifference(position_str:str,destination_str:str) -> tuple:
    ''' 
        Provides the longitudal and latitudal differnce in meters
        This calculation is suitable for distances 10km or less and
        it is customized to mitigate the poor floating point precision of Micropython.
        For larger distances the Haversine formula would be needed
    '''

    lat_p = convert_dd_int(position_str[0])
    lon_p = convert_dd_int(position_str[1])

    lat_d = convert_dd_int(destination_str[0])
    lon_d = convert_dd_int(destination_str[1])

    dy = lat_d-lat_p # delta latitude
    dx = lon_d-lon_p # delta longitude

    # correct dx due to the curvature of the earth
    latA = float(destination_str[0])
    latB = float(position_str[0])
    latAvg = (latA+latB)/2
    correct_dx = dx*cos(radians(latAvg))   

    # The distances dx and dy are arcdegrees, which is then converted to meters
    dx_meters = correct_dx * 0.011112       # 111120 / 10000000 # 111120 meters in an arcdegree
    dy_meters = dy * 0.011112           	# 111120 / 10000000 # 111120 meters in an arcdegree

    return dx_meters, dy_meters

In [7]:
positionDifference(position,destination)

(718.719241236055, -1121.900856)

In [8]:
def distancebearing(position_str:str,destination_str:str) -> tuple:
    '''
    Provides distance (meters:float) and bearing (degrees:float) from a position to a destination 
    '''

    dLon, dLat = positionDifference(position_str,destination_str)
    
    distance = round(sqrt(dLon**2 + dLat**2),1)   # distance in meters, limted to 1 decimal
    bearing = round(atan2(dLon,dLat),3) # bearing in radians, limited to 3 decimals

    return distance, degrees(bearing)

In [9]:
distancebearing(position,destination)

(1332.4, 147.36474490764775)

In [ ]:
async def control_position():
    """Main control loop for position holding"""
    print("Position control started...")
    
    last_time = time.time()

    kp = 0.1
    ki = 0.1
    kd = 0

    integral_error_x =0
    integral_error_y = 0

    last_error_x = 0
    last_error_y = 0
    
    n= 20
    while n >= 0:
    
        n=n-1


        # Calculate position error in meters
        error_x, error_y = positionDifference(position,destination)
        
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
   
        # Calculate PID control values
        # Proportional term
        p_term_x = kp * error_x
        p_term_y = kp * error_y
        
        # Integral term
        integral_error_x += error_x * dt
        integral_error_y += error_y * dt
        i_term_x = ki * integral_error_x
        i_term_y = ki * integral_error_y
        
        # Derivative term
        derivative_x = (error_x - last_error_x) / dt if dt > 0 else 0
        derivative_y = (error_y - last_error_y) / dt if dt > 0 else 0
        d_term_x = kd * derivative_x
        d_term_y = kd * derivative_y
        
        # Calculate thrust commands
        thrust_x = p_term_x + i_term_x + d_term_x
        thrust_y = p_term_y + i_term_y + d_term_y
        
        # Store errors for next iteration
        last_error_x = error_x
        last_error_y = error_y
        
    
        distance_meters = round(sqrt(thrust_x**2 + thrust_y**2),1)   # distance in meters, limted to 1 decimal
        desiredcourse = round(atan2(thrust_x,thrust_y),3) # bearing in radians, limited to 3 decimals

        # Reduce the surge as the RoboBouy approached the waypoint
        # It slows down inverse square to the holdRadius
        vmax = 1
        holdgain = 1
        holdRadius = 10

        surge = min(vmax, holdgain * (distance_meters**2 / holdRadius**2) )


        print(f"Position error: {error_x:.2f}m, {error_y:.2f}m, Applying thrust: surge={surge:.2f}, desiredcourse={degrees(desiredcourse):.2f}")
        
        await asyncio.sleep(0.01)  # Control frequency

In [12]:
async def main():
   print("Main was called")
   asyncio.create_task(control_position())

asyncio.run(main())

Main was called


Position control started...
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -1121.90m, Applying thrust: surge=1.00, desiredcourse=147.36
Position error: 718.72m, -